In [1]:
using Revise

includet("../GameAnalysis.jl")
includet("../SimplexPlots.jl")

In [3]:
function mixture_set_intersection(s1, s2; max_diff=1e-2)
    shared = zeros(size(s1,1), min(size(s1,2),size(s2,2)))
    count = 0
    for i in 1:size(s1,2)
        m1 = s1[:,i]
        for j in 1:size(s2,2)
            m2 = s2[:,j]
            if maximum(m1 .- m2) < max_diff
                count += 1
                shared[:,count] = m1
            end
        end
    end
    return shared[:,1:count]
end

mixture_set_intersection (generic function with 1 method)

In [47]:
players = 100
actions = 3
functions = 200
gmg = gaussian_mixture_game(players, actions, functions)
fine_grid = mixture_grid(actions, 89)
coarse_grid = simplex_project(mixture_grid(actions, 13), .99)
regret_grid = regret(gmg, fine_grid);
simplex_heatmap(fine_grid, regret_grid)
plot!(savefig="../figures/gmg_regret.png")


kpathsea: Running mktextfm phvr8t
/usr/share/texlive/texmf-dist/web2c/mktexnam: Could not map source abbreviation  for phvr8t.
/usr/share/texlive/texmf-dist/web2c/mktexnam: Need to update ?
mktextfm: Running mf-nowin -progname=mf \mode:=ljfour; mag:=1; nonstopmode; input phvr8t
This is METAFONT, Version 2.7182818 (TeX Live 2019/Debian) (preloaded base=mf)

kpathsea: Running mktexmf phvr8t

! I can't find file `phvr8t'.
<*> ...:=ljfour; mag:=1; nonstopmode; input phvr8t
                                                  
Please type another input file name
! Emergency stop.
<*> ...:=ljfour; mag:=1; nonstopmode; input phvr8t
                                                  
Transcript written on mfput.log.
grep: phvr8t.log: No such file or directory
mktextfm: `mf-nowin -progname=mf \mode:=ljfour; mag:=1; nonstopmode; input phvr8t' failed to make phvr8t.tfm.
kpathsea: Appending font creation commands to missfont.log.

kpathsea: Running mktextfm phvr8t
/usr/share/texlive/texmf-dist/web2c/

In [48]:
reg_thresh = 1e-4
rd_trace = logged_replicator_dynamics(gmg, coarse_grid; offset=-1000, iters=10000);
rd_endpoints = rd_trace[:,:,end]
rd_equilibria = filter_unique(filter_regrets(gmg, rd_endpoints, threshold=reg_thresh), max_diff=0.02) 
println("Replicator Dynamics found ", size(rd_equilibria, 2), " equilibria:")
display(round.(rd_equilibria, digits=2))
println("Regrets: ", round.(regret(gmg, rd_equilibria), sigdigits=3))

gd_trace = logged_gain_descent(gmg, coarse_grid, step_size=[1e-5 * .99^i for i in 1:10000], iters=10000)
gd_endpoints = gd_trace[:,:,end]
gd_equilibria = filter_unique(filter_regrets(gmg, gd_endpoints, threshold=reg_thresh), max_diff=0.02) 
println("Gradient Descent found ", size(gd_equilibria, 2), " equilibria:")
display(round.(gd_equilibria, digits=2))
println("Regrets: ", round.(regret(gmg, gd_equilibria), sigdigits=3))

shared_equilibria = mixture_set_intersection(rd_equilibria, gd_equilibria)
display(round.(shared_equilibria, digits=2))

3×5 Matrix{Float64}:
 0.17  0.97  0.0   0.28  0.03
 0.49  0.03  0.12  0.72  0.9
 0.34  0.0   0.88  0.0   0.06

3×9 Matrix{Float64}:
 0.27  0.05  0.97  0.77  0.08  0.28  0.28  0.17  0.0
 0.6   0.93  0.03  0.06  0.62  0.72  0.45  0.49  0.38
 0.13  0.03  0.0   0.17  0.3   0.0   0.27  0.34  0.62

3×3 Matrix{Float64}:
 0.17  0.97  0.28
 0.49  0.03  0.72
 0.34  0.0   0.0

Replicator Dynamics found 5 equilibria:
Regrets: [5.19e-6, 4.33e-7, 7.3e-6, 0.0, 4.14e-5]
Gradient Descent found 9 equilibria:
Regrets: [7.73e-6, 1.14e-6, 4.33e-7, 1.43e-5, 1.03e-5, 8.57e-6, 1.68e-5, 3.38e-5, 4.76e-5]


In [50]:
simplex_heatmap(fine_grid, regret_grid, show=false)

for i in 1:size(coarse_grid,2)
    simplex_scatter!(gd_trace[:,i,2:1000], markerfacecolor=:yellow, marker=:p, markersize=.05, show=false)
end
for i in 1:size(coarse_grid,2)
    simplex_scatter!(rd_trace[:,i,2:1000], marker=:d, markerfacecolor=:red, markersize=.05, show=false)
end

simplex_scatter!(coarse_grid, marker=:p, markersize=.1, markerfacecolor=:black)

simplex_scatter!(rd_equilibria, marker=:p, markersize=.2, markerfacecolor=:red, show=false)
simplex_scatter!(gd_equilibria, marker=:p, markersize=.2, markerfacecolor=:yellow, show=false)
simplex_scatter!(shared_equilibria, marker=:p, markersize=.2, markerfacecolor=:orange, show=false)



plot!(savefig="../figures/nash_traces.png")


kpathsea: Running mktextfm phvr8t
/usr/share/texlive/texmf-dist/web2c/mktexnam: Could not map source abbreviation  for phvr8t.
/usr/share/texlive/texmf-dist/web2c/mktexnam: Need to update ?
mktextfm: Running mf-nowin -progname=mf \mode:=ljfour; mag:=1; nonstopmode; input phvr8t
This is METAFONT, Version 2.7182818 (TeX Live 2019/Debian) (preloaded base=mf)

kpathsea: Running mktexmf phvr8t

! I can't find file `phvr8t'.
<*> ...:=ljfour; mag:=1; nonstopmode; input phvr8t
                                                  
Please type another input file name
! Emergency stop.
<*> ...:=ljfour; mag:=1; nonstopmode; input phvr8t
                                                  
Transcript written on mfput.log.
grep: phvr8t.log: No such file or directory
mktextfm: `mf-nowin -progname=mf \mode:=ljfour; mag:=1; nonstopmode; input phvr8t' failed to make phvr8t.tfm.
kpathsea: Appending font creation commands to missfont.log.

kpathsea: Running mktextfm phvr8t
/usr/share/texlive/texmf-dist/web2c/

In [ ]:
reg_thresh = 1e-4
gd_trace = logged_gain_descent(gmg, coarse_grid, step_size=[1e-5 * .99^i for i in 1:10000], iters=10000)
gd_endpoints = gd_trace[:,:,end]
gd_equilibria = filter_unique(filter_regrets(gmg, gd_endpoints, threshold=reg_thresh), max_diff=0.02) 
println("Gradient Descent found ", size(gd_equilibria, 2), " equilibria:")
display(round.(gd_equilibria, digits=2))
println("Regrets: ", round.(regret(gmg, gd_equilibria), sigdigits=3))

In [ ]:
simplex_heatmap(fine_grid, regret_grid, show=false, title="Gradient Descent")
for i in 1:size(coarse_grid,2)
    simplex_scatter!(gd_trace[:,i,2:1000], marker=:p, markersize=.075, show=false)
end
simplex_scatter!(gd_trace[:,:,1], marker=:p, markersize=.1, markerfacecolor=:purple)
for i in 1:size(coarse_grid,2)
    if regret(g,g, gd_trace[:,i,1001]) >= reg_thresh
        simplex_scatter!(gd_trace[:,i,1001], marker=:p, markersize=.15, markerfacecolor=:red, show=false)
    end
end
for i in 1:size(coarse_grid,2)
    if regret(gmg, gd_trace[:,i,1001]) < reg_thresh
        simplex_scatter!(gd_trace[:,i,1001], marker=:p, markersize=.2, markerfacecolor=:green, show=false)
    end
end
plot!(show=true)

In [ ]:
reg_thresh = 1e-3
fp_trace = logged_fictitious_play(gag, coarse_grid; initial_weight=50, iters=1000000);
fp_endpoints = fp_trace[:,:,end]
fp_equilibria = filter_unique(filter_regrets(gag, fp_endpoints, threshold=reg_thresh), max_diff=0.02) 
println("Fictitious Play found ", size(fp_equilibria, 2), " equilibria:")
display(round.(fp_equilibria, digits=2))
println("Regrets: ", round.(regret(gag, fp_equilibria), sigdigits=3))

In [ ]:
simplex_heatmap(fine_grid, regret_grid, show=false, title="Fictitious Play")
for i in 1:size(coarse_grid,2)
    simplex_scatter!(fp_trace[:,i,2:1000], marker=:p, markersize=.075, show=false)
end
simplex_scatter!(fp_trace[:,:,1], marker=:p, markersize=.1, markerfacecolor=:purple)
for i in 1:size(coarse_grid,2)
    if regret(gag, fp_trace[:,i,1001]) >= reg_thresh
        simplex_scatter!(fp_trace[:,i,1001], marker=:p, markersize=.15, markerfacecolor=:red, show=false)
    end
end
for i in 1:size(coarse_grid,2)
    if regret(gag, fp_trace[:,i,1001]) < reg_thresh
        simplex_scatter!(fp_trace[:,i,1001], marker=:p, markersize=.2, markerfacecolor=:green, show=false)
    end
end
simplex_scatter!(fp_equilibria, marker=:p, markersize=.2, markerfacecolor=:green, show=true)

In [ ]:
reg_thresh = 1e-2
ibr_trace = logged_iterated_better_response(gag, coarse_grid; iters=1000000, step_size=2e-4);
ibr_endpoints = ibr_trace[:,:,end]
ibr_equilibria = filter_unique(filter_regrets(gag, ibr_endpoints, threshold=reg_thresh), max_diff=0.02) 
println("Iterated Better Response found ", size(ibr_equilibria, 2), " equilibria:")
display(round.(ibr_equilibria, digits=2))
println("Regrets: ", round.(regret(gag, ibr_equilibria), sigdigits=3))

In [ ]:
simplex_heatmap(fine_grid, regret_grid, show=false, title="Iterated Better Response")
for i in 1:size(coarse_grid,2)
    simplex_scatter!(ibr_trace[:,i,2:1000], marker=:p, markersize=.075, show=false)
end
simplex_scatter!(ibr_trace[:,:,1], marker=:p, markersize=.1, markerfacecolor=:purple)
for i in 1:size(coarse_grid,2)
    if regret(gag, ibr_trace[:,i,1001]) >= reg_thresh
        simplex_scatter!(ibr_trace[:,i,1001], marker=:p, markersize=.15, markerfacecolor=:red, show=false)
    end
end
for i in 1:size(coarse_grid,2)
    if regret(gag, ibr_trace[:,i,1001]) < reg_thresh
        simplex_scatter!(ibr_trace[:,i,1001], marker=:p, markersize=.2, markerfacecolor=:green, show=false)
    end
end
simplex_scatter!(ibr_equilibria, marker=:p, markersize=.2, markerfacecolor=:green, show=true)